In [1]:
import pandas as pd
import re
import numpy as np
from collections import Counter

# Load the Hyundai CSV file
hyundai_data = pd.read_csv('/home/bipin/Documents/kotak/car_model/manufacturer_data/HYUNDAI.csv')
print(f"Loaded {len(hyundai_data)} records from HYUNDAI.csv")
hyundai_data.head()

# Function to clean and normalize model names
def clean_model(text):
    if pd.isna(text) or text is None:
        return ""
    # Convert to string
    text = str(text).upper()
    # Remove common prefixes and company names
    text = re.sub(r'HYUNDAI\s+MOTOR\s+INDIA\s+LTD,?\s*', '', text)
    text = re.sub(r'HYUNDAI\s+MOTOR\s+INDIA\s*', '', text) 
    text = re.sub(r'HYUNDAIMOT,?\s*', '', text)
    text = re.sub(r'HYUNDAI\s+MOTORS,?\s*', '', text)
    text = re.sub(r'HYUNDAI\s*', '', text)
    # Remove special characters and extra spaces
    text = re.sub(r'[^\w\s-]', ' ', text)
    text = re.sub(r'\s+', ' ', text).strip()
    return text

# Create a list of cleaned model names
hyundai_data['clean_model'] = hyundai_data['rc_maker_model'].apply(clean_model)

# Filter out too generic or empty model names
generic_terms = ['LTD', 'BS', 'IV', 'III', 'BS-IV', 'BS-III', 'BSIV', 'BSIII', 'STD', 'LIMITED']
filtered_models = hyundai_data['clean_model'].apply(
    lambda x: x if x and all(term not in x.split() for term in generic_terms) else np.nan
)

# Count model name frequencies
model_counts = Counter([m for models in filtered_models.dropna() for m in models.split()])
model_counts = {k: v for k, v in model_counts.items() if v >= 10}  # Keep only frequent terms

# Print top model keywords by frequency
print("Top model keywords by frequency:")
sorted_counts = sorted(model_counts.items(), key=lambda x: x[1], reverse=True)
for model, count in sorted_counts[:30]:
    print(f"{model}: {count}")

# Define primary Hyundai model keywords
hyundai_model_keywords = [
    # Popular sedans
    "VERNA", "ACCENT", "ELANTRA", "AURA", "XCENT", "SONATA",
    
    # Hatchbacks
    "I10", "I20", "EON", "SANTRO", "GRAND", "GETZ", "NIOS", 
    
    # SUVs and Crossovers
    "CRETA", "VENUE", "TUCSON", "KONA", "ALCAZAR", "TERRACAN", "SANTA", "FE",
    
    # Variants/Special models
    "ACTIVE", "SPORTZ", "MAGNA", "ASTA", "ELITE", "NEO", "ERA", "DLITE",
    
    # Engine/Trim specifiers
    "CRDI", "VTVT", "KAPPA", "GDI", "SX", "GLS", "GLE", "DCT", "AMT", "IVT",
    
    # Common model combinations
    "GRAND I10", "I20 ACTIVE", "I20 SPORTZ", "I20 MAGNA", "I20 ASTA",
    "SANTRO XO", "SANTRO XL", "SANTRO XP", "SANTRO XG", "SANTRO GL",
    "I10 SPORTZ", "I10 MAGNA", "I10 ERA", "I10 ASTA",
    "VERNA VTVT", "VERNA CRDI", "VERNA FL",
    "CRETA VTVT", "CRETA CRDI",
    "GRAND NIOS", "I10 NIOS"
]

# Create alias map for common misspellings and variations
hyundai_alias_map = {
    # i10 variations
    "I-10": "I10",
    "110": "I10",
    "I 10": "I10",
    "I.10": "I10",
    "1.10": "I10",
    "1 10": "I10",
    "1-10": "I10",
    "10": "I10",
    "IL0": "I10",
    "II0": "I10",
    "ILO": "I10",
    "I1O": "I10",
    
    # i20 variations
    "I-20": "I20",
    "120": "I20",
    "I 20": "I20",
    "I.20": "I20",
    "1.20": "I20",
    "1 20": "I20",
    "1-20": "I20",
    "20": "I20",
    "I.2O": "I20",
    "I 2O": "I20",
    
    # Santro variations
    "SANTO": "SANTRO",
    "SANTR": "SANTRO",
    "SENTRO": "SANTRO",
    "SNTRO": "SANTRO",
    "SANTRAO": "SANTRO",
    "SANTORO": "SANTRO",
    "SAMTRO": "SANTRO",
    "SANTR0": "SANTRO",
    "SANTRA": "SANTRO",
    "SANTROO": "SANTRO",
    "CENTRO": "SANTRO",
    
    # Verna variations
    "VENA": "VERNA",
    "VERENA": "VERNA",
    "VRNA": "VERNA",
    "VIRNA": "VERNA",
    "VARNAH": "VERNA",
    "VERNA FL": "VERNA",
    
    # Creta variations
    "CREATA": "CRETA",
    "KRETA": "CRETA",
    
    # Grand variations
    "GAND": "GRAND",
    "GRANND": "GRAND",
    "GRAN": "GRAND",
    "GRAAND": "GRAND",
    "GARND": "GRAND",
    
    # Elantra variations
    "ELENTRA": "ELANTRA",
    "ELANDRA": "ELANTRA",
    
    # Eon variations
    "ECON": "EON",
    "EON*": "EON",
    
    # Trim level variations
    "MAGMA": "MAGNA",
    "MANGA": "MAGNA",
    "MEGNA": "MAGNA",
    "MAHGNA": "MAGNA",
    "MANGNA": "MAGNA",
    
    "ASTRA": "ASTA",
    "ASATA": "ASTA",
    "ASTAA": "ASTA",
    
    # Common combinations mapped to base models
    "I10 SPORTZ": "I10",
    "I10 MAGNA": "I10", 
    "I10 ASTA": "I10",
    "I10 ERA": "I10",
    
    "I20 SPORTZ": "I20",
    "I20 MAGNA": "I20",
    "I20 ASTA": "I20",
    "I20 ELITE": "I20",
    "I20 ACTIVE": "I20",
    
    "XCENT VTVT": "XCENT",
    "XCENT CRDI": "XCENT",
    
    "SANTRO XL": "SANTRO",
    "SANTRO XK": "SANTRO", 
    "SANTRO XO": "SANTRO",
    "SANTRO XG": "SANTRO",
    "SANTRO GL": "SANTRO",
    "SANTRO GLS": "SANTRO",
    "SANTRO XING": "SANTRO",
    "SANTRO GING": "SANTRO",
    
    "GRAND I10": "GRAND I10",
    "GRAND I-10": "GRAND I10",
    "GRAND 1-10": "GRAND I10",
    "GRAND 1 10": "GRAND I10",
    "GRAND I 10": "GRAND I10",
    
    "VERNA CRDI": "VERNA",
    "VERNA VTVT": "VERNA",
}

# Function to normalize text
def normalize(text):
    # Handle NaN/None values or any non-string types
    if pd.isna(text) or text is None:
        return ""
    # Convert to string if it's not already a string
    if not isinstance(text, str):
        text = str(text)
    # Then proceed with normalization
    return re.sub(r'[^A-Z0-9 ]', '', text.upper())

# Function to extract Hyundai model
def extract_hyundai_model(text):
    txt = normalize(text)
    # First, correct aliases
    for bad, good in hyundai_alias_map.items():
        if bad in txt:
            txt = txt.replace(bad, good)
    
    # Then search for any keyword
    for kw in sorted(hyundai_model_keywords, key=len, reverse=True):
        if kw in txt:
            return kw
    return "NOT FOUND"

# Apply extraction to the entire dataset
hyundai_data['hyundai_model'] = hyundai_data['rc_maker_model'].apply(extract_hyundai_model)

# Display extraction results
model_freq = hyundai_data['hyundai_model'].value_counts()
print("\nModel extraction results:")
print(model_freq.head(20))
print(f"\nModels found: {(hyundai_data['hyundai_model'] != 'NOT FOUND').sum()} out of {len(hyundai_data)} ({(hyundai_data['hyundai_model'] != 'NOT FOUND').sum()/len(hyundai_data)*100:.1f}%)")

# Create a final_model column using fallback logic
hyundai_data['final_model'] = hyundai_data['hyundai_model'].apply(lambda x: x if x != "NOT FOUND" else "NOT FOUND")

# Save results to CSV for review
output_path = '/home/bipin/Documents/kotak/car_model/clean_model/hyundai_models_extracted.csv'
hyundai_data.to_csv(output_path, index=False)
print(f"\nSaved results to {output_path}")

# Create a frequency mapping of original values to extracted models
mapping_df = hyundai_data.groupby('rc_maker_model')['hyundai_model'].agg(
    lambda x: x.value_counts().index[0] if len(x.value_counts()) > 0 else "NOT FOUND"
).reset_index()
mapping_df.columns = ['original_value', 'mapped_model']
mapping_df['count'] = mapping_df['original_value'].map(hyundai_data['rc_maker_model'].value_counts())
mapping_df = mapping_df.sort_values('count', ascending=False)

# Save the mapping for review
mapping_path = '/home/bipin/Documents/kotak/car_model/clean_model/hyundai_model_mapping.csv'
mapping_df.to_csv(mapping_path, index=False)
print(f"Saved mapping to {mapping_path}")

# Save the keywords and alias map for reuse
with open('/home/bipin/Documents/kotak/car_model/clean_model/hyundai_keywords.py', 'w') as f:
    f.write("hyundai_model_keywords = [\n")
    for kw in sorted(hyundai_model_keywords):
        f.write(f"    \"{kw}\",\n")
    f.write("]\n\n")
    
    f.write("hyundai_alias_map = {\n")
    for bad, good in sorted(hyundai_alias_map.items()):
        f.write(f"    \"{bad}\": \"{good}\",\n")
    f.write("}\n")

print("Keywords and alias map saved to hyundai_keywords.py")

Loaded 18686 records from HYUNDAI.csv
Top model keywords by frequency:
1: 7246
CRDI: 2435
2: 1980
I: 1871
SANTRO: 1722
O: 1686
SX: 1666
I10: 1625
I20: 1518
MAGNA: 1499
SPORTZ: 1469
VERNA: 1386
ASTA: 1362
6: 1294
GRAND: 1210
VTVT: 1200
10: 1043
CRETA: 881
4: 862
S: 855
20: 808
BS4: 777
GLS: 726
NEW: 687
GL: 605
ERA: 605
E3: 570
MT: 526
AT: 523
5: 508

Model extraction results:
hyundai_model
SANTRO       2167
VERNA        2156
I10          2060
I20          1778
GRAND        1672
CRETA        1162
MAGNA         975
XCENT         829
SPORTZ        794
CRDI          746
VTVT          621
ASTA          454
EON           418
NOT FOUND     332
ACCENT        321
GETZ          282
ELANTRA       279
VENUE         275
DLITE         192
ELITE         147
Name: count, dtype: int64

Models found: 18354 out of 18686 (98.2%)

Saved results to /home/bipin/Documents/kotak/car_model/clean_model/hyundai_models_extracted.csv
Saved mapping to /home/bipin/Documents/kotak/car_model/clean_model/hyundai_model_m